In [3]:
import tensorflow as tf

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [196]:
def customOps(n):
    
    customOps.mat = tf.placeholder(tf.float32, shape = [n, n])    
    
    # 1.Transpose the elements in the bottom-right triangle of A

    '''
    A_transpose             : transpose of matrix A
    A_rev                   : flip left to right of matrix A
    A_transpose_rev         : flip left to right of A_transpose
    upper                   : upper triangular matrix of A_rev
    lower                   : lower triangular matrix of A_transpose_rev
    upper_rev               : flip left to right of upper
    lower_rev               : flip left to right of lower
    diagonal                : diagonal matrix of A_rev
    diagonal_rev            : flip left to right of diagonal
    B                       : Transpose the elements in the bottom-right triangle of A  
    '''

#     A_transpose = tf.transpose(customOps.mat)                         
#     A_rev = tf.reverse(customOps.mat, [-1])   
    
#     A_transpose_rev = tf.reverse(A_transpose, [-1])             
    
#     upper =  tf.matrix_band_part(A_rev, 0, -1)                  
#     lower = tf.matrix_band_part(A_transpose_rev, -1, 0)
    
#     upper_rev = tf.reverse(upper, [-1])
#     lower_rev = tf.reverse(lower, [-1])
    
    diagonal = tf.matrix_band_part(tf.reverse(tf.transpose(customOps.mat), [-1]), 0, 0)  # diagonal matrix
    diagonal_rev = tf.reverse(tf.matrix_band_part(tf.reverse(tf.transpose(customOps.mat), [-1]), 0, 0), [-1])
    
    upper_left_part = tf.reverse(tf.matrix_band_part(tf.reverse(customOps.mat, [-1]), 0, -1),[-1])
    bottom_right_part =  tf.reverse(tf.matrix_band_part(tf.reverse(tf.transpose(customOps.mat), [-1]), -1, 0), [-1])
    
#     A = upper_rev + lower_rev - diagonal_rev
    A = upper_left_part + bottom_right_part - diagonal_rev

    # 2.Take the maximum value along the columns of A to get a vector m~ 
    
    m = tf.reduce_max(A, axis = 1)
    
    
    # 3.softmax matrix
    # dont forget to convert the data type to float before use
    temp = []
    for i in range(n):
        padding = [[0,i]]
        temp.append(tf.pad(tf.nn.softmax(tf.slice(m,[0],[n-i]), axis = 0), padding, mode='CONSTANT', name=None))
    B = tf.convert_to_tensor(temp)
        
    # 4.Sum along the rows of B to obtain vector ~v 1 
    
    v1 = tf.reduce_sum(B, axis = 1)
    
    
    # Sum along the columns of B to get another vector ~v 2 
    
    v2 = tf.reduce_sum(B, axis = 0)
    
    
    # 5.Concatenate the two vectors and take a softmax of this vector: ~v = softmax(concat(~v 1 , ~v 2 ))
    
    v  = tf.nn.softmax(tf.concat([v1, v2], axis = 0), axis = 0)
    
    
    # 6.Get the index number in vector ~v with maximum value
    
    index = tf.argmax(v, axis = 0)
    
    
    # 7.If index number is greater than n/3 : 
    #      finalVal = ||v 1 − ~v 2 || 2  
    # else :
    #      finalval = ||v 1 + ~v 2 || 2
    
    finalVal = tf.cond(index > tf.cast((n/3),tf.int64), lambda : tf.square(v1-v2), lambda : tf.square(v1+v2))
    
    
    return finalVal


In [197]:
import numpy as np

if __name__ == '__main__':
    mat = np.asarray([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])
    n = mat.shape[0]

    finalVal = customOps(n)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    outVal = sess.run(finalVal, feed_dict={customOps.mat: mat})
    print(outVal)
    sess.close()

[3.8131134e-09 1.2882569e-01 5.8674649e-04 1.1206542e-01]


In [74]:
with tf.Session() as sess:
    mat = np.asarray([[1., 2., 3., 4.],
                      [5., 6., 7., 8.],
                      [9., 10., 11., 12.],
                      [13., 14., 15., 16.]])
    
#     mask_a = tf.matrix_band_part(mat, 0, -1) # Upper triangular matrix of 0s and 1s
#     rev = tf.reverse(mask_a, [-1])
    rev1 = tf.reverse(mat, [-1])
    mask_a1 = tf.matrix_band_part(rev1, 0, -1) # Upper triangular matrix of 0s and 1s
    rev2 = tf.reverse(mask_a1, [-1])
#     mask_b = tf.matrix_band_part(B, 0, 0)  # Diagonal matrix of 0s and 1s
#     mask = tf.cast(mask_a - mask_b, dtype=tf.bool) # Make a bool mask

#     upper_triangular_flat = tf.boolean_mask(B, mask)
    
#     print(sess.run(mat))
    print(sess.run(rev2))    

[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  0.]
 [ 9. 10.  0.  0.]
 [13.  0.  0.  0.]]


In [75]:
# m1 = [1,2,3,4]
# a = tf.stack([m1 for i in range(10)] )
# rev1 = tf.reverse(mat, [-1])
# mask_a1 = tf.matrix_band_part(rev1, 0, -1) # Upper triangular matrix of 0s and 1s
# rev2 = tf.reverse(mask_a1, [-1])
# # B = tf.nn.softmax(rev2, axis = 0)
m = tf.reduce_max(mat, axis = 1)
B = tf.reshape(tf.tile(m,[4]), [4,4])
C = tf.nn.softmax(B, axis = 1)

with tf.Session() as sess:
#     print(sess.run(rev2))
    print(sess.run(B))    
    print(sess.run(C))        

[[ 4.  8. 12. 16.]
 [ 4.  8. 12. 16.]
 [ 4.  8. 12. 16.]
 [ 4.  8. 12. 16.]]
[[6.03167786e-06 3.29318453e-04 1.79801783e-02 9.81684472e-01]
 [6.03167786e-06 3.29318453e-04 1.79801783e-02 9.81684472e-01]
 [6.03167786e-06 3.29318453e-04 1.79801783e-02 9.81684472e-01]
 [6.03167786e-06 3.29318453e-04 1.79801783e-02 9.81684472e-01]]


In [69]:
# customOps.A = mat    
    
# # 1.Transpose the elements in the bottom-right triangle of A 
# B = tf.transpose(customOps.A)
# rev_upper = tf.reverse(customOps.A, [-1])
# rev_lower = tf.reverse(B, [-1])
# upper =  tf.matrix_band_part(rev_upper, 0, -1)
# lower = tf.matrix_band_part(rev_lower, -1, 0)
# final_upper = tf.reverse(upper, [-1])
# final_lower = tf.reverse(lower, [-1])
# diagonal = tf.matrix_band_part(rev_lower, 0, 0)
# rev_diagonal = tf.reverse(diagonal, [-1])
# val = final_upper + final_lower - rev_diagonal

# n = 3
# m = tf.reduce_max(B, axis = 1)
# b = tf.reshape(tf.tile(m,[n]),[n,n])
# # b = tf.tile(m,[n])

# with tf.Session() as sess:
#     print(sess.run(customOps.A))
#     print(sess.run(upper))
#     print(customOps.A)
#     print(sess.run(rev_upper))
#     print(sess.run(B))
#     print(sess.run(rev_lower))
#     print(sess.run(final_upper))
#     print(sess.run(final_lower))   
#     print(sess.run(diagonal))
#     print(sess.run(m))
    
#     print(sess.run(b))

In [121]:
tmp = [[1,2,3],
      [1,2,3]]
tmp1 = [1,2,3]
paddings = [[0,1],[0,1]]
pad = [[0,2]]
val = tf.pad(tmp1, pad, mode='CONSTANT', name=None)
with tf.Session() as sess:
    print(sess.run(val))

[1 2 3 0 0]


In [122]:
mat = [1.,2.,3.,4.]
# sof = tf.slice(mat,[0],[2])
sof = tf.pad(tf.nn.softmax(tf.slice(mat,[0],[2]), axis = 0), pad, mode='CONSTANT', name=None)
with tf.Session() as sess:
    print(sess.run(sof))

[0.26894143 0.7310586  0.         0.        ]


In [163]:
mat = np.asarray([[1., 2., 3., 4.],
                      [5., 6., 7., 8.],
                      [9., 10., 11., 12.],
                      [13., 14., 15., 16.]])
m = tf.reduce_max(mat, axis = 1)
with tf.Session() as sess:

#     temp = tf.ones([4,4],dtype=tf.float32)
    temp  =[]
    for i in range(0,4):
        pads = [[0,i]]
        sof = tf.slice(m,[0],[4-i])
        
#         temp[0:] = tf.pad(tf.nn.softmax(tf.slice(m,[0],[4-i]), axis = 0), pads, mode='CONSTANT', name=None)

        temp.append(tf.pad(tf.nn.softmax(tf.slice(m,[0],[4-i]), axis = 0), pads, mode='CONSTANT', name=None))
#         temp = tf.reshape(temp, [1,4])
        tmp = tf.convert_to_tensor(temp)
    print(sess.run((tmp)))


[[6.03167786e-06 3.29318453e-04 1.79801783e-02 9.81684472e-01]
 [3.29320439e-04 1.79802867e-02 9.81690393e-01 0.00000000e+00]
 [1.79862100e-02 9.82013790e-01 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]


In [151]:
mat = np.asarray([[1., 2., 3., 4.],
                      [5., 6., 7., 8.],
                      [9., 10., 11., 12.],
                      [13., 14., 15., 16.]])
m = tf.reduce_max(mat, axis = 1)
with tf.Session() as sess:
    for i in range(3):
        m.append()
    

Tensor("Max_78:0", shape=(4,), dtype=float64)
